In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.insert(1, '/Users/andriisutiagin/Jobb/programering/ftx_pandas_report')
from util.date_helper import DateHelper as dh
from get_price_ccxt import GetPriceCcxt

ModuleNotFoundError: No module named 'ccxt'

In [42]:
date = '2022-11'

stamps = dh.get_month_range_date(date)

start_time = stamps['start_time_string']
end_time = stamps['end_time_string']

In [43]:
ccxt = GetPriceCcxt()

In [23]:
def set_price(contract=None):
    if 'USDT' in contract:
        contract = contract.split('USDT')[0]+'/USD'
    else:
        print('This is not USDT contract')
    try:
        ohlc = ccxt.get_price(market = contract, time_s = stamps['end_time_timestamp'])
    except:
        raise Exception('Fail to get price from ccxt')
    return ohlc[4]

In [45]:
#Bybit-Derivatives-TradeHistory-20221101-20221202
df = pd.read_csv('Bybit-Derivatives-TradeHistory-20221101-20221202.csv')
df = df.loc[(df['Transaction Time(UTC+0)'] >= start_time) & \
            (df['Transaction Time(UTC+0)'] <= end_time)].copy()
print('Start date', df['Transaction Time(UTC+0)'].min())
print('End date', df['Transaction Time(UTC+0)'].max())

Start date 2022-11-19 14:46:58
End date 2022-11-30 22:56:24


In [11]:
df.head(5)

,Contracts,Order No.,Transaction Time(UTC+0),Leverage,Direction,Order Type,Filled Type,Filled Qty,Filled Price,Order Price,Trading Fee Rate,Fees Paid,Transaction ID
12,DOGEUSDT,29c9e7bc,2022-11-30 22:56:24,10,Open Short,Limit,Trade,125000.0,0.10768,0.10768,0.00006,0.807600,eee06177
13,DOGEUSDT,af6de876,2022-11-30 18:04:58,10,Close Short,Limit,Trade,35042.0,0.10033,0.10033,0.00006,0.210946,68d0082b
14,DOGEUSDT,af6de876,2022-11-30 18:04:58,10,Close Short,Limit,Trade,1946.0,0.10033,0.10033,0.00006,0.011715,d59e1ee6
15,DOGEUSDT,af6de876,2022-11-30 18:04:58,10,Close Short,Limit,Trade,18851.0,0.10033,0.10033,0.00006,0.113479,a9ad58cd
16,DOGEUSDT,af6de876,2022-11-30 18:04:58,10,Close Short,Limit,Trade,22514.0,0.10033,0.10033,0.00006,0.135530,a107bc0b


In [12]:
df['usd_value'] = df['Filled Qty'] * df['Filled Price']

In [13]:
df_fund = df.loc[df['Filled Type'].isin(['Funding'])]
df_fund.head()

,Contracts,Order No.,Transaction Time(UTC+0),Leverage,Direction,Order Type,Filled Type,Filled Qty,Filled Price,Order Price,Trading Fee Rate,Fees Paid,Transaction ID,usd_value
23,DOGEUSDT,851-Sell,2022-11-30 16:00:00,10,Open Short,--,Funding,1875000.0,0.10417,0.00000,-0.000100,-19.531875,73d3f9dc,195318.75
29,DOGEUSDT,851-Sell,2022-11-30 08:00:00,10,Open Short,--,Funding,2000000.0,0.10617,0.00000,-0.000093,-19.654190,402b0549,212340.00
30,DOGEUSDT,851-Sell,2022-11-30 00:00:00,10,Open Short,--,Funding,2000000.0,0.10138,0.00000,0.000105,21.381042,07c67cb1,202760.00
31,DOGEUSDT,851-Sell,2022-11-29 16:00:00,10,Open Short,--,Funding,2000000.0,0.10153,0.00000,-0.000100,-20.306000,fe515489,203060.00
32,DOGEUSDT,851-Sell,2022-11-29 08:00:00,10,Open Short,--,Funding,2000000.0,0.10240,0.00000,0.003059,626.511872,31f94274,204800.00


In [35]:
df_trade = df.loc[df['Filled Type'].isin(['Trade'])]
buy_sell_df = df_trade.groupby(['Direction'])['usd_value'].sum()

In [36]:
buy_sell_df

Direction
Close Long      48352.57135
Close Short    642710.46020
Open Long       46361.62350
Open Short     846270.01455
Name: usd_value, dtype: float64

In [37]:
sell = buy_sell_df['Close Long'] +  buy_sell_df['Open Short']
sell

894622.5859

In [38]:
buy = buy_sell_df['Open Long'] +  buy_sell_df['Close Short']
buy

689072.0837

In [44]:
group_by_size = df_trade.groupby(['Contracts', 'Direction'])[['Filled Qty', 'usd_value']].sum()
group_by_size = group_by_size.reset_index()
group_by_size

,Contracts,Direction,Filled Qty,usd_value
0,CHZUSDT,Close Long,125000.0,29402.57135
1,CHZUSDT,Close Short,995000.0,219279.29670
2,CHZUSDT,Open Long,125000.0,28981.25000
3,CHZUSDT,Open Short,995000.0,236435.01455
4,DOGEUSDT,Close Long,200000.0,18950.00000
5,DOGEUSDT,Close Short,2063315.0,199772.12650
6,DOGEUSDT,Open Long,186685.0,17380.37350
7,DOGEUSDT,Open Short,3675000.0,373955.00000
8,LTCUSDT,Close Short,3000.0,223659.03700
9,LTCUSDT,Open Short,3000.0,235880.00000


In [46]:
group_by_size[group_by_size['Contracts'] == 'DOGEUSDT']

,Contracts,Direction,Filled Qty,usd_value
4,DOGEUSDT,Close Long,200000.0,18950.0000
5,DOGEUSDT,Close Short,2063315.0,199772.1265
6,DOGEUSDT,Open Long,186685.0,17380.3735
7,DOGEUSDT,Open Short,3675000.0,373955.0000


In [17]:
rest = []

for market in group_by_size['Contracts'].unique():
    close_short = (group_by_size['Contracts'] == market) & (group_by_size['Direction'] == 'Close Short')
    open_short = (group_by_size['Contracts'] == market) & (group_by_size['Direction'] == 'Open Short')
    close_long = (group_by_size['Contracts'] == market) & (group_by_size['Direction'] == 'Close Long')
    open_long = (group_by_size['Contracts'] == market) & (group_by_size['Direction'] == 'Open Long')
    
    rest_long = group_by_size[close_long]['Filled Qty'].sum() - group_by_size[open_long]['Filled Qty'].sum()
    rest_short = group_by_size[close_short]['Filled Qty'].sum() - group_by_size[open_short]['Filled Qty'].sum()
    
    rest += [{'Contracts': market, 'Rest Direction': 'Long', 'Size': rest_long}]
    rest += [{'Contracts': market, 'Rest Direction': 'Short', 'Size': rest_short}]

In [19]:
rest_pd = pd.DataFrame(rest)
rest_pd

,Contracts,Rest Direction,Size
0,CHZUSDT,Long,0.0
1,CHZUSDT,Short,0.0
2,DOGEUSDT,Long,13315.0
3,DOGEUSDT,Short,-1611685.0
4,LTCUSDT,Long,0.0
5,LTCUSDT,Short,0.0


In [28]:
rest_pd['Price'] = rest_pd.apply(lambda r: set_price(r['Contracts']) if r['Size'] !=0 else 0.0, axis=1)
rest_pd['Usd_value'] = rest_pd['Price'] * rest_pd['Size']

In [34]:
rest_pd = rest_pd.groupby('Contracts').agg({'Usd_value':'sum'})
rest_pd

,Usd_value
Contracts,
CHZUSDT,0.0
DOGEUSDT,-511478.4
LTCUSDT,0.0


In [39]:
fees = df_trade['Fees Paid'].sum()
fees

269.01608058

In [40]:
funding_payment = - df_fund['Fees Paid'].sum()
funding_payment

-479.75277053